In [1]:
# задача с конкурса Tinkoff: https://boosters.pro/championship/tinkoff1/overview
#Датасет содержит данные о кредитах на покупку электроники, которые были одобрены Tinkoff.ru. 
# Необходимо предсказать, выберет ли покупатель кредит от Tinkoff.ru
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [9]:
# записываем CSV-файл в объект DataFrame
df = pd.read_csv('credit_train3.csv', encoding='ANSI', sep=';')

In [10]:
# смотрим на первые пять строк
df.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,"59998,00",10,1.6,"0,770249",GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,"10889,00",6,1.1,"0,248514",GRD,МОСКВА,43000.0,2.0,0.0,0
2,3,M,32,MAR,SPC,"10728,00",12,1.1,"0,459589",SCH,ОБЛ САРАТОВСКАЯ,23000.0,5.0,0.0,0
3,4,F,27,DIV,SPC,"12009,09",12,1.1,"0,362536",GRD,ОБЛ ВОЛГОГРАДСКАЯ,17000.0,2.0,0.0,0
4,5,M,45,MAR,SPC,"16908,89",10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,25000.0,1.0,0.0,0


In [11]:
#проверяем типы данных и заполненность столбцов
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   client_id             170746 non-null  int64  
 1   gender                170746 non-null  object 
 2   age                   170746 non-null  int64  
 3   marital_status        170746 non-null  object 
 4   job_position          170746 non-null  object 
 5   credit_sum            170746 non-null  object 
 6   credit_month          170746 non-null  int64  
 7   tariff_id             170746 non-null  float64
 8   score_shk             170746 non-null  object 
 9   education             170746 non-null  object 
 10  living_region         170554 non-null  object 
 11  monthly_income        170745 non-null  float64
 12  credit_count          161516 non-null  float64
 13  overdue_credit_count  161516 non-null  float64
 14  open_account_flg      170746 non-null  int64  
dtype

In [12]:
#удаляем столбец с номером клиента (так как он незначимый) 
# и с регионом проживания (так как он нуждается в серьезной предобработке)
df.drop(['client_id', 'living_region'], axis=1, inplace=True)

In [13]:
# анализируем зависимую переменную: какие значения она принимает и сколько раз
df['open_account_flg'].value_counts(dropna=False)

0    140690
1     30056
Name: open_account_flg, dtype: int64

In [14]:
# считаем, какая точность (доля правильных ответов) была бы у модели, если всем подряд пресказывать, что кредит они не выберут
d=140690/(140690+30056)
d

0.823972450306303

In [15]:
# дозаполняем числовые столбцы с пропусками медианными значениями
df['age'].fillna(df['age'].median(), inplace=True)
df['credit_count'].fillna(df['credit_count'].median(), inplace=True)
df['overdue_credit_count'].fillna(df['overdue_credit_count'].median(), inplace=True)

In [17]:
#меняем в столбцах 'credit_sum', 'score_shk'  запятые на точки  и преобразуем их в числовой  формат
for i in ['credit_sum', 'score_shk']:
    df[i] = df[i].str.replace(',', '.').astype('float')

In [18]:
# дозаполняем ставшие теперь числовыми столбцы 'credit_sum', 'score_shk'   медианными значениями
df['score_shk'].fillna(df['score_shk'].median(), inplace=True)
df['monthly_income'].fillna(df['monthly_income'].median(), inplace=True)
df['credit_sum'].fillna(df['credit_sum'].median(), inplace=True)

In [19]:
# смотрим, что получилось
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170746 non-null  int64  
 2   marital_status        170746 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170746 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170746 non-null  float64
 8   education             170746 non-null  object 
 9   monthly_income        170746 non-null  float64
 10  credit_count          170746 non-null  float64
 11  overdue_credit_count  170746 non-null  float64
 12  open_account_flg      170746 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 16.9+ MB


In [22]:
# кодируем пол методом факторизации
codes, uniques = pd.factorize(np.array(df['gender']))
df['gender'] = codes

In [23]:
# кодируем нечисловые столбцы методом дамми-кодирования
ata = pd.concat([df, 
                 pd.get_dummies(df['job_position'], prefix="job_position"), 
                 pd.get_dummies(df['education'], prefix="education"), 
                 pd.get_dummies(df['marital_status'], prefix="marital_status")], 
                 axis=1)

In [24]:
#удаляем старые нечисловые столбцы, вместо них уже появились новые числовые
df.drop(['job_position','education','marital_status'], axis=1, inplace=True)

In [25]:
# указываем входы и выход модели
y = df['open_account_flg']
X = df.drop(('open_account_flg'), axis=1)

In [26]:
# формируем из набора данных тестовую и обучающую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

In [27]:
#попробуем использовать градиентный бустинг, реализованный в библиотеке sklearn
from sklearn import ensemble
gbt = ensemble.GradientBoostingClassifier(n_estimators = 150,  max_depth=5,  random_state=21)
gbt.fit(X_train, y_train)
gbt.score(X_test, y_test)

0.8295525534905513